<a href="https://colab.research.google.com/github/mandananaderi/14th-Group/blob/main/Retail_Sales_Forecasting_Week1_Mandana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# تمام مراحل در یک سلول
import os
import zipfile
from google.colab import files

# آپلود فایل kaggle.json
uploaded = files.upload()

# تنظیم محیط و دسترسی‌ها
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# نصب کتابخانه Kaggle
!pip install -q kaggle

# دانلود دیتاست
!kaggle competitions download -c rossmann-store-sales

# اکسترکت فایل‌ها (اگر فایل زیپ شده باشد)
with zipfile.ZipFile('rossmann-store-sales.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

# نمایش فایل‌های دانلود شده
!ls

Saving kaggle.json to kaggle.json
rossmann-store-sales.zip: Skipping, found more recently modified local copy (use --force to force download)
 drive					  sample_data		  test.csv
'Report of Rossmann Store Sales_1.html'   sample_submission.csv   train.csv
 rossmann-store-sales.zip		  store.csv


In [55]:
from google.colab import drive
import shutil
import os

# اتصال به گوگل درایو
drive.mount('/content/drive')

# ایجاد پوشه اختصاصی
drive_path = "/content/drive/My Drive/Colab Datasets/Rossmann"
os.makedirs(drive_path, exist_ok=True)

# لیست فایل‌هایی که می‌خواهید ذخیره کنید
files_to_save = ['train.csv', 'test.csv', 'store.csv', 'sample_submission.csv']

# کپی فایل‌ها به گوگل درایو
for file in files_to_save:
    if os.path.exists(file):
        shutil.copy(file, os.path.join(drive_path, file))
        print(f"File {file} saved to Google Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File train.csv saved to Google Drive
File test.csv saved to Google Drive
File store.csv saved to Google Drive
File sample_submission.csv saved to Google Drive


In [56]:
import os
from google.colab import drive, userdata

# 1. وصل به درایو
drive.mount('/content/drive')

# 2. تنظیم توکن (یکی از روش‌ها را انتخاب کنید)
token_path = "/content/drive/My Drive/.kaggle/kaggle.json"
if os.path.exists(token_path):
    !mkdir -p ~/.kaggle
    !cp "{token_path}" ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
else:
    print("لطفاً فایل kaggle.json را در درایو ذخیره کنید")

# 3. دسترسی به دیتاست‌ها
data_path = "/content/drive/My Drive/Colab Datasets/"
if os.path.exists(f"{data_path}train.csv"):
    train = pd.read_csv(f"{data_path}train.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
لطفاً فایل kaggle.json را در درایو ذخیره کنید


In [57]:
!ls -l "/content/drive/My Drive/kaggle.json"

-rw------- 1 root root 69 May 14 20:10 '/content/drive/My Drive/kaggle.json'


In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Initialization

In [58]:
!pip install ydata_profiling

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport


# Data Loading

In [61]:
# Read train

df_train = pd.read_csv("train.csv")
df_train
df_train.info()

<ipython-input-61-1acec2463b4e>:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("train.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [62]:
#Read store

df_store = pd.read_csv("store.csv")
df_store
df_store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [63]:
# Read test

df_te = pd.read_csv("test.csv")
df_te

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0
...,...,...,...,...,...,...,...,...
41083,41084,1111,6,2015-08-01,1.0,0,0,0
41084,41085,1112,6,2015-08-01,1.0,0,0,0
41085,41086,1113,6,2015-08-01,1.0,0,0,0
41086,41087,1114,6,2015-08-01,1.0,0,0,0


In [64]:
# merge two datasets with store

df_test = pd.merge(df_store, df_te, on="Store")

df = pd.merge(df_store, df_train, on="Store")
df

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,5,2015-07-31,5263,555,1,1,0,1
1,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,4,2015-07-30,5020,546,1,1,0,1
2,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,3,2015-07-29,4782,523,1,1,0,1
3,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2,2015-07-28,5011,560,1,1,0,1
4,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,1,2015-07-27,6102,612,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",6,2013-01-05,4771,339,1,0,0,1
1017205,1115,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",5,2013-01-04,4540,326,1,0,0,1
1017206,1115,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",4,2013-01-03,4297,300,1,0,0,1
1017207,1115,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec",3,2013-01-02,3697,305,1,0,0,1


#EDA

In [65]:
# EDA

profile_report = ProfileReport(df, title="EDA of Rossmann Store Sales")
profile_report.to_file("Report of Rossmann Store Sales_1.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 18/18 [00:17<00:00,  1.03it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
# Comparison of sales on days with and without promotions

df.groupby("Promo")["Sales"].mean()

df.groupby("Promo2")["Sales"].mean()


,Sales
Promo2,
0,6124.710161
1,5423.718218


## Data Cleaning

In [34]:
# Input and Target separation

y_train = df.Sales
X_train = df.drop('Sales', axis=1)


In [53]:
# Removing variables with missing values ​​greater than 20% and very low correlation with the target variable

X_train_NomissVar = X_train.drop(['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], axis=1)
X_train_NomissVar


,Store,StoreType,Assortment,CompetitionDistance,Promo2,DayOfWeek,Date,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,c,a,1270.0,0,5,2015-07-31,555,1,1,0,1
1,1,c,a,1270.0,0,4,2015-07-30,546,1,1,0,1
2,1,c,a,1270.0,0,3,2015-07-29,523,1,1,0,1
3,1,c,a,1270.0,0,2,2015-07-28,560,1,1,0,1
4,1,c,a,1270.0,0,1,2015-07-27,612,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,d,c,5350.0,1,6,2013-01-05,339,1,0,0,1
1017205,1115,d,c,5350.0,1,5,2013-01-04,326,1,0,0,1
1017206,1115,d,c,5350.0,1,4,2013-01-03,300,1,0,0,1
1017207,1115,d,c,5350.0,1,3,2013-01-02,305,1,0,0,1


In [52]:
# Recategorizing and Encoding StoreType and Assortment

X_train_NomissVar['StoreType'] = X_train_NomissVar['StoreType'].replace({'a': '1', 'd': '2', 'c':'3', 'b': '3'})
X_train_NomissVar['Assortment'] = X_train_NomissVar['Assortment'].replace({'a': '1', 'c': '2', 'b':'2'})


def frequency_table(variable):
    unique_elements, counts = np.unique(variable, return_counts=True)
    percentages = (counts/len(variable))* 100

    for i,j,k in zip(unique_elements, counts, percentages):
        print(f"{i}: count: {j}, percentage: {k:.2f}%")

    return



recategorize_var = ['StoreType', 'Assortment']

for var in recategorize_var:
    print(frequency_table(X_train_NomissVar[var]))


1: count: 551627, percentage: 54.23%
2: count: 312912, percentage: 30.76%
3: count: 152670, percentage: 15.01%
None
1: count: 537445, percentage: 52.84%
2: count: 479764, percentage: 47.16%
None
